In [18]:
# imports
import numpy as np
import scipy as scp
from scipy.sparse.linalg import spsolve
from scipy import sparse
from scipy.sparse.linalg import splu
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
from time import time
import scipy.stats as ss
from functools import partial
from scipy.integrate import quad
import scipy as scp
from math import factorial

In probability theory and statistics, a diffusion process is a solution to a stochastic differential equation. It is a continuous-time Markov process with almost surely continuous sample paths. Brownian motion, reflected Brownian motion and Ornstein–Uhlenbeck processes are examples of diffusion processes.

# Black Scholes

In [14]:
# vars
# r=0                   # int rt
# sig=0                 # diffusion coefficient
# S0=0                  # current price of stock
# K=0                   # strike price
# T=0                   # Maturity (yrs)
rvexp=None            # function to generate solution of GBM
price=0
S_vec = None          # vector for the stock
price_vec = None      # vector for the price
mesh = None           # for plotting purposes
ex = None             # exercise the option
payoff = None         # payoff val

In [3]:
# payoff function
def payoff_function(S):
    if payoff=='call':
        fPayoff = np.maximum(S-K,0)
    elif payoff=='put':
        fPayoff = np.maximum(K-S,0)
    return fPayoff

In [13]:
# closed formula ish
def BS(payoff,S0,K,T,r,sigma):
    d1 = (np.log(S0/K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - (sigma * np.sqrt(T))

    if payoff=="call":
        return S0 * ss.norm.cdf( d1 ) - K * np.exp(-r * T) * ss.norm.cdf( d2 )
    elif payoff=="put":
        return K * np.exp(-r * T) * ss.norm.cdf( -d2 ) - S0 * ss.norm.cdf( -d1 )
    else:
        raise ValueError("invalid type. Set 'call' or 'put'")


## Fourier Inversion

Price is obtained by inversion of the characteristic function. cf_normal is the characteristic function for normal distributions

In [5]:
def cf_normal(u,mu=1,sig=2):
    return np.exp(1j*u*mu-0.5*u**2*sig**2)

In [9]:
def Q1(k, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the stock numeraire.
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( (np.exp(-u*k*1j) / (u*1j)) * 
                                  cf(u-1j) / cf(-1.0000000000001j) )  
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=2000 )[0]


def Q2(k, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the money market numeraire
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( np.exp(-u*k*1j) /(u*1j) * cf(u) )
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=2000 )[0]

In [10]:
def fourier_inv():
    k = np.log(K/S0)
    cf_GBM = partial(cf_normal,mu=(r-0.5*sig**2)*T,sig=sig*np.sqrt(T))
#     function binding
    if payoff=='call':
        call=S0*Q1(k,cf_GBM,np.inf)-K*np.exp(-r*T)*Q2(k,cf_GBM,np.inf)
#         pricing func
        return call
    elif payoff=='put':
        put=K*np.exp(-r*T)*(1-Q2(k,cf_GBM,np.inf))-S0*(1-Q1(k,cf_GBM,np.inf))
#         pricing func
    else:
            raise ValueError("invalid type. Set 'call' or 'put'")

## Running the functions

In [17]:
S0=100.0
K=100.0
T=1.0
r=0.1
sig=0.2
a_call=BS('call',S0,K,T,r,sig)
a_put=BS('put',S0,K,T,r,sig)
print(a_call)
print(a_put)

13.269676584660893
3.753418388256833


## Monte Carlo

In [20]:
np.random.seed(seed=44)              # seed the rand num gen
N=10000000                           # Number of RVs

W = ss.norm.rvs( (r-0.5*sig**2)*T , np.sqrt(T)*sig, N)
S_T = S0 * np.exp(W)

call = np.mean( np.exp(-r*T) * np.maximum(S_T-K,0) )
put = np.mean( np.exp(-r*T) * np.maximum(K-S_T,0) )
call_err = ss.sem( np.exp(-r*T) * np.maximum(S_T-K,0) )  # standard error
put_err = ss.sem( np.exp(-r*T) * np.maximum(K-S_T,0) )   # standard error

In [21]:
print("Call price: {}, with error: {}".format(call, call_err))
print("Put price: {}, with error: {}".format(put, put_err))

Call price: 13.26333800663666, with error: 0.005093638687208466
Put price: 3.7553894006350093, with error: 0.002214066662789331


In [25]:
payoff='call'
fourier_inv()
# 13.26967658466063

13.26967658466063